## Random Forest Classifier with Pipelines

In [30]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [31]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [32]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [34]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time'] = encoder.fit_transform(df['time'])

In [35]:
# Divide dataset into independent and dependent features

In [36]:
X = df.drop(labels = ['time'], axis = 1)
y = df['time']

In [37]:
from sklearn.model_selection import train_test_split

In [50]:
X_train, X_test, y_train , y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# Handling missing values automatically
from sklearn.impute import SimpleImputer
# We have nominal values in dataset so we can convert them using OneHotEncoding
from sklearn.preprocessing import OneHotEncoder
# Feature Scaling
from sklearn.preprocessing import StandardScaler
# Automating the process
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [52]:
categorical_cols = ['sex','smoker', 'day']
numerical_cols = ['total_bill', 'tip', 'size']

In [53]:
## Automating the feature engineering

In [54]:
# Pipeline for numerical values
num_pipeline =  Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy= 'median')),# Handles the missing values
        ('scaler', StandardScaler()) # Feature scaling
    ] 
) 


# Pipeline for categorical values
cat_pipeline = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy= 'most_frequent')),# Handles the missing values
        ('onehotencoder', OneHotEncoder()) # encoding features
    ]
)

In [55]:
# Creating a wrapper for pipeline
preprocessor = ColumnTransformer([
            ('num_pipeline', num_pipeline, numerical_cols),
            ('cat_pipeline', cat_pipeline, categorical_cols)

])

In [56]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [63]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [64]:
# Automating the model training process
models = {
    'Random Forest' : RandomForestClassifier(),
    'Decision Tree' : DecisionTreeClassifier(),
    'SVC Classifier' : SVC()

}

In [65]:
from sklearn.metrics import accuracy_score

In [66]:
def evaluate_model(X_train,y_train, X_test,y_test, models ):
    
    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        #Training the model
        model.fit(X_train, y_train)
        
        
        #Predicting the result
        y_pred = model.predict(X_test)
        
        #Get the accuracy
        
        test_model_score = accuracy_score(y_pred, y_test)
        
        report[list(models.keys())[i]] = test_model_score
        
             
        
    return report   
        
    
    
    

In [67]:
evaluate_model(X_train,y_train, X_test,y_test, models)

{'Random Forest': 0.9591836734693877,
 'Decision Tree': 0.9387755102040817,
 'SVC Classifier': 0.9591836734693877}

In [68]:
classifier = RandomForestClassifier()

In [76]:
params = {'max_depth' : [3,4,5,6,7,8,9,10],
         'n_estimators' : [50,100, 200, 300],
         'criterion' : ['gini', 'entropy']
         }

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
rcv = RandomizedSearchCV(estimator= classifier, param_distributions=params, cv = 5, scoring = 'accuracy' , verbose = 3)
rcv.fit(X_train, y_train)

In [78]:
rcv.best_params_

{'n_estimators': 100, 'max_depth': 9, 'criterion': 'entropy'}